<picture>
  <source media="(prefers-color-scheme: dark)" srcset="https://vespa.ai/assets/vespa-ai-logo-heather.svg">
  <source media="(prefers-color-scheme: light)" srcset="https://vespa.ai/assets/vespa-ai-logo-rock.svg">
  <img alt="#Vespa" width="200" src="https://vespa.ai/assets/vespa-ai-logo-rock.svg" style="margin-bottom: 25px;">
</picture>

# Feeding performance

This explorative notebook intends to shine some light on the different modes of feeding documents to Vespa.
We will look at these different methods:

1. Sync feeding from pyVespa.
2. Async feeding from pyVespa.
3. Using `app.feed_iterable()` from pyVespa.
4. Using Vespa CLI[TODO:Link].


<div class="alert alert-info">
    Refer to <a href="https://pyvespa.readthedocs.io/en/latest/troubleshooting.html">troubleshooting</a>
    for any problem when running this guide.
</div>


[Install pyvespa](https://pyvespa.readthedocs.io/) and start Docker Daemon, validate minimum 6G available:


In [1]:
#!pip3 install pyvespa
#!docker info | grep "Total Memory"

## Create an application package

The [application package](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.ApplicationPackage)
has all the Vespa configuration files -
create one from scratch:


In [9]:
%load_ext autoreload
%autoreload 2

from vespa.package import (
    ApplicationPackage,
    Field,
    Schema,
    Document,
    FieldSet,
)

package = ApplicationPackage(
    name="pyvespafeed",
    schema=[
        Schema(
            name="doc",
            document=Document(
                fields=[
                    Field(name="id", type="string", indexing=["summary"]),
                    Field(name="text", type="string", indexing=["summary"]),
                ]
            ),
            fieldsets=[FieldSet(name="default", fields=["text"])],
        )
    ],
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Note that the name cannot have `-` or `_`.


## Deploy the Vespa application

Deploy `package` on the local machine using Docker,
without leaving the notebook, by creating an instance of
[VespaDocker](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.deployment.VespaDocker). `VespaDocker` connects
to the local Docker daemon socket and starts the [Vespa docker image](https://hub.docker.com/r/vespaengine/vespa/).

If this step fails, please check
that the Docker daemon is running, and that the Docker daemon socket can be used by clients (Configurable under advanced settings in Docker Desktop).


In [10]:
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker()
app = vespa_docker.deploy(application_package=package)

Waiting for configuration server, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 10/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Application is up!
Finished deployment.


`app` now holds a reference to a [Vespa](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.application.Vespa) instance.


## Feeding documents to Vespa

In this example we use the [HF Datasets](https://huggingface.co/docs/datasets/index) library to stream the
[BeIR/nfcorpus](https://huggingface.co/datasets/BeIR/nfcorpus) dataset and index in our newly deployed Vespa instance. Read
more about the [NFCorpus](https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/):

> NFCorpus is a full-text English retrieval data set for Medical Information Retrieval.

The following uses the [stream](https://huggingface.co/docs/datasets/stream) option of datasets to stream the data without
downloading all the contents locally. The `map` functionality allows us to convert the
dataset fields into the expected feed format for `pyvespa` which expects a dict with the keys `id` and `fields`:

`{ "id": "vespa-document-id", "fields": {"vespa_field": "vespa-field-value"}}`


In [3]:
from datasets import load_dataset

dataset = load_dataset(
    "Cohere/wikipedia-2023-11-embed-multilingual-v3",
    "ext",
    split="train",
    streaming=False,
)
vespa_feed = dataset.map(
    lambda x: {"id": x["_id"] + "-iter", "fields": {"text": x["text"]}}
)
vespa_json_feed = vespa_feed.map(
    lambda x: {
        "put": f"id:pyvespa-feed:doc::{x['_id']}-json",
        "fields": {"text": x["text"]},
    }
)

Map:   0%|          | 0/10967 [00:00<?, ? examples/s]

In [4]:
vespa_feed_iter = vespa_feed.select_columns(["id", "fields"]).to_list()

In [5]:
from vespa.io import VespaResponse, VespaQueryResponse


def callback(response: VespaResponse, id: str):
    if not response.is_successful():
        print(f"Error when feeding document {id}: {response.get_json()}")


app.feed_iterable(
    vespa_feed_iter, schema="doc", namespace="pyvespa-feed", callback=callback
)

## Feeding async


In [ ]:
import asyncio

loop = asyncio.get_event_loop()
loop.create_task(
app.feed_iterable_improved(
    vespa_feed_iter, schema="doc", namespace="pyvespa-feed", callback=callback
)

<Task pending name='Task-5' coro=<Vespa.feed_iterable_improved() running at /Users/thomas/Repos/pyvespa/vespa/application.py:523>>

Feeding documents ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:-- ?

In [21]:
from vespa.feeder import VespaFeeder

feeder = VespaFeeder(
    endpoint="http://localhost:8080/document/v1/pyvespa-feed/doc/docid/",
    documents=vespa_feed_iter,
)

In [22]:
import requests
import json

# Set up the URL and the data payload
url = "http://localhost:8080/document/v1/pyvespa-feed/doc/docid/1"
data = {"fields": {"text": "This is a test"}}

# Use requests to send the POST request
response = requests.post(
    url, headers={"Content-Type": "application/json"}, data=json.dumps(data)
)

# Print out the response content to see the result
print(response.text)

{"pathId":"/document/v1/pyvespa-feed/doc/docid/1","id":"id:pyvespa-feed:doc::1"}


In [23]:
vespa_feed_iter[0]

{'id': '20231101.ext_246_0-iter',
 'fields': {'text': "Güiquipedia Ayúa Zona prevas Índizi A–Z El conceju La troji Embassy Help for non-Extremaduran speakers Ayuda para quienes no hablan estremeñu'''"}}

In [24]:
feeder.feed_sync()

Output()

KeyboardInterrupt: 

In [103]:
feeder_list = vespa_json_feed.select_columns(["put", "fields"]).to_list()

In [104]:
curl -X POST -H "Content-Type:application/json" --data '
  {
      "fields": {
          "artist": "Coldplay",
          "album": "A Head Full of Dreams",
          "year": 2015
      }
  }' \
  http://localhost:8080/document/v1/mynamespace/music/docid/a-head-full-of-dreams


TypeError: get_document_v1_path() missing 1 required positional argument: 'id'

In [ ]:
feeder = VespaFeeder(
    en

In [99]:
with app.asyncio() as vespa_async:
    for feed in vespa_json_feed:
        vespa_async.feed(feed, schema="doc", namespace="pyvespa-feed")

AttributeError: __enter__

In [ ]:
with VespaAsync()

## Feeding with Vespa CLI


In [95]:
vespa_json_feed.select_columns(["put", "fields"]).to_json(
    "vespa_feed.json", orient="records", lines=True
)

Creating json from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

4505289

In [96]:
!head -3 vespa_feed.json

{"put":"id:pyvespa-feed:doc::20231101.ext_246_0-json","fields":{"text":"G\u00fciquipedia Ay\u00faa Zona prevas \u00cdndizi A\u2013Z El conceju La troji Embassy Help for non-Extremaduran speakers Ayuda para quienes no hablan estreme\u00f1u'''"}}
{"put":"id:pyvespa-feed:doc::20231101.ext_246_1-json","fields":{"text":"|class=\"MainPageBG\" style=\"width: 50%; border: 1px solid #006600; background-color: #F5FBEF; vertical-align: top; -moz-border-radius:10px; font-size:90% \" |"}}
{"put":"id:pyvespa-feed:doc::20231101.ext_247_0-json","fields":{"text":".com (el ingr\u00e9s commercial, comercial) es un domi\u00f1u d'internet hen\u00e9ricu que horma parti el sistema e domi\u00f1us d'internet. El domi\u00f1u .com es unu los domi\u00f1us orihinalis d'internet, hue estableciu en Eneru e 1985 i oga\u00f1u es llevau pola compa\u00f1ia VeriSign."}}


Now we can feed to Vespa using `feed_iterable` which accepts any `Iterable` and an optional callback function where we can
check the outcome of each operation. The application is configured to use [embedding](https://docs.vespa.ai/en/embedding.html)
functionality, that produce a vector embedding using a concatenation of the title and the body input fields. This step is computionally expensive. Read more
about embedding inference in Vespa in the [Accelerating Transformer-based Embedding Retrieval with Vespa](https://blog.vespa.ai/accelerating-transformer-based-embedding-retrieval-with-vespa/).


In [97]:
!vespa feed vespa_feed.json

{
  "feeder.operation.count": 10967,
  "feeder.seconds": 1.274,
  "feeder.ok.count": 10967,
  "feeder.ok.rate": 8605.321,
  "feeder.error.count": 0,
  "feeder.inflight.count": 0,
  "http.request.count": 10967,
  "http.request.bytes": 3295053,
  "http.request.MBps": 2.585,
  "http.exception.count": 0,
  "http.response.count": 10967,
  "http.response.bytes": 1380468,
  "http.response.MBps": 1.083,
  "http.response.error.count": 0,
  "http.response.latency.millis.min": 84,
  "http.response.latency.millis.avg": 126,
  "http.response.latency.millis.max": 266,
  "http.response.code.counts": {
    "200": 10967
  }
}


### Plain Semantic Search

The following uses dense vector representations of the query and the document and matching is performed and accelerated by Vespa's support for
[approximate nearest neighbor search](https://docs.vespa.ai/en/approximate-nn-hnsw.html).
The vector embedding representation of the text is obtained using Vespa's [embedder functionality](https://docs.vespa.ai/en/embedding.html#embedding-a-query-text).


In [7]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding,q)) limit 5",
        query=query,
        ranking="semantic",
        body={"input.query(q)": f"embed({query})"},
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-5072  Lycopene-rich treatments modify noneosinophili...
1  MED-2472  Vegan regimen with reduced medication in the t...
2  MED-2464  Low vegetable intake is associated with allerg...
3  MED-2458  Manipulating antioxidant intake in asthma: a r...
4  MED-2450  Protective effect of fruits, vegetables and th...


### Hybrid Search

This is one approach to combine the two retrieval strategies and where we use Vespa's support for
[cross-hits feature normalization and reciprocal rank fusion](https://docs.vespa.ai/en/phased-ranking.html#cross-hit-normalization-including-reciprocal-rank-fusion). This
functionality is exposed in the context of `global` re-ranking, after the distributed query retrieval execution which might span 1000s of nodes.

#### Hybrid search with the OR query operator

This combines the two methods using logical disjunction (OR). Note that the first-phase expression in our `fusion` expression is only using the semantic score, this
because usually semantic search provides better recall than sparse keyword search alone.


In [8]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding,q)) limit 5",
        query=query,
        ranking="fusion",
        body={"input.query(q)": f"embed({query})"},
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-2464  Low vegetable intake is associated with allerg...
1  MED-2450  Protective effect of fruits, vegetables and th...
2  MED-2458  Manipulating antioxidant intake in asthma: a r...
3  MED-2461  The association of diet with respiratory sympt...
4  MED-5069  From beans to berries and beyond: teamwork bet...


#### Hybrid search with the RANK query operator


This combines the two methods using the [rank](https://docs.vespa.ai/en/reference/query-language-reference.html#rank) query operator. In this case
we express that we want to retrieve the top-1000 documents using vector search, and then have sparse features like BM25 calculated as well (second operand
of the rank operator). Finally the hits are re-ranked using the reciprocal rank fusion


In [9]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql="select * from sources * where rank({targetHits:1000}nearestNeighbor(embedding,q), userQuery()) limit 5",
        query=query,
        ranking="fusion",
        body={"input.query(q)": f"embed({query})"},
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-2464  Low vegetable intake is associated with allerg...
1  MED-2450  Protective effect of fruits, vegetables and th...
2  MED-2458  Manipulating antioxidant intake in asthma: a r...
3  MED-2461  The association of diet with respiratory sympt...
4  MED-5069  From beans to berries and beyond: teamwork bet...


#### Hybrid search with filters

In this example we add another query term to the yql, restricting the nearest neighbor search to only consider documents that have vegetable in the title.


In [10]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql='select * from sources * where title contains "vegetable" and rank({targetHits:1000}nearestNeighbor(embedding,q), userQuery()) limit 5',
        query=query,
        ranking="fusion",
        body={"input.query(q)": f"embed({query})"},
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-2464  Low vegetable intake is associated with allerg...
1  MED-2450  Protective effect of fruits, vegetables and th...
2  MED-3199  Potential risks resulting from fruit/vegetable...
3  MED-2085  Antiplatelet, anticoagulant, and fibrinolytic ...
4  MED-4496  The effect of fruit and vegetable intake on ri...


## Cleanup


In [8]:
vespa_docker.container.stop()
vespa_docker.container.remove()

NotFound: 404 Client Error for http+docker://localhost/v1.41/containers/4e603c787fdfd34cdebcd1dc7638f86e9275bca89c74f1f5e1b89b6058f18ad5/stop: Not Found ("no container with name or ID "4e603c787fdfd34cdebcd1dc7638f86e9275bca89c74f1f5e1b89b6058f18ad5" found: no such container")

## Next steps

This is just an intro into the capabilities of Vespa and pyvespa.
Browse the site to learn more about schemas, feeding and queries -
find more complex applications in
[examples](https://pyvespa.readthedocs.io/en/latest/examples.html).
